In [13]:
import numpy as np
import inspect
from scipy.interpolate import interp1d
from IPython.display import display
from ipywidgets import interact_manual, Layout, FloatProgress
from atmos_cloud_sim_uj_utils import show_plot
from PySDM import Formulae
from PySDM.physics import si
from PySDM.physics.freezing_temperature_spectrum import niemand_et_al_2012
from PySDM.physics.freezing_temperature_spectrum import bigg_1953
from PySDM.physics.heterogeneous_ice_nucleation_rate import abifm
from PySDM.initialisation.spectra import Lognormal

from PySDM_examples.UIUC_2021 import make_particulator, run_simulation, \
    make_sampling_plot, make_temperature_plot, make_freezing_spec_plot, make_pdf_plot

In [17]:
import matplotlib
font = {'family' : 'monospace',
        'weight' : 'light',
        'size'   : 15
       }
matplotlib.rc('font', **font)  

In [18]:
bigg_1953.DT_median = 36.5
niemand_et_al_2012.a = -0.517
niemand_et_al_2012.b = 8.934
abifm.m = 54.48
abifm.c = -10.67

In [19]:
SEP = ';'
    
progbar = FloatProgress(value=0, min=0, max=100, description='%')
display(progbar)

@interact_manual(
    Shima_T_fz=[('Niemand_et_al_2012'), ('Bigg_1953')],
    interpolation=['linear', 'quadratic', 'cubic'],
    times_s = f'000{SEP} 500{SEP} 750{SEP} 4500{SEP} 5000',
    temps_K = f'255{SEP} 235{SEP} 245{SEP} 245{SEP} 225',
    log2_n_sd=(4, 14), 
    n_steps=(20, 100),
    ensemble_n=(1, 9),
    drop_v_um3=(1, 5),
    ln10_drop_n=(15, 20),
    ln10_vol_m3=(4, 8),
    m_mode_um2=((.5, 4.5)),
    ln_s_geom=(0, 3.)
)
def interface(*, 
              Shima_T_fz,
              times_s, temps_K, interpolation, 
              log2_n_sd, n_steps, ensemble_n,
              drop_v_um3,
              ln10_drop_n,
              ln10_vol_m3,
              m_mode_um2,
              ln_s_geom
             ):
    output_widget = globals()[inspect.currentframe().f_code.co_name].widget.children[-1]
    output_widget.layout = Layout(visibility = 'hidden')
    progbar.value = 0
    
    temperature_profile = interp1d(
        x=np.asarray(times_s.split(SEP), dtype=float), 
        y=np.asarray(temps_K.split(SEP), dtype=float),
        kind=interpolation
    )
    assert temperature_profile.x[0] == 0

    A_spec = Lognormal(norm_factor=1, m_mode=m_mode_um2*si.um**2, s_geom=np.exp(ln_s_geom))
    
    params = {
        'droplet_volume': drop_v_um3 * si.um**3,
        'total_particle_number': 10**ln10_drop_n,
        'volume': 10**ln10_vol_m3
    }
    
    output = []
    for singular in (True, False):
        for seed in np.arange(ensemble_n): 
            particulator = make_particulator(
                n_sd=2**log2_n_sd, 
                dt=temperature_profile.x[-1]/n_steps,
                initial_temperature = temperature_profile(0),
                singular=singular,
                seed=seed,
                shima_T_fz=Shima_T_fz,
                ABIFM_spec=A_spec,
                **params
            )
            output.append({
                **run_simulation(particulator, temperature_profile, n_steps),
                'singular': singular
            })
            progbar.value += 100/2/ensemble_n
    make_sampling_plot(output)
    show_plot('fig0.pdf')
    make_temperature_plot(output)
    show_plot('fig1.pdf')
    
    formulae = Formulae(freezing_temperature_spectrum=Shima_T_fz)

    make_freezing_spec_plot(
        output, 
        formulae, 
        surf_spec=A_spec,
        **params
    )
    show_plot('fig2.pdf')
    make_pdf_plot(
        A_spec,
        formulae.freezing_temperature_spectrum.pdf,
        A_range = (0 * si.um ** 2, 5 * si.um ** 2),
        T_range = (min(temperature_profile.y), max(temperature_profile.y))
     )
    show_plot('fig3.pdf')
    output_widget.layout = Layout(visibility = 'visible')
    
button = interface.widget.children[-2]
button.description = 'rerun and replot'
button.click()

FloatProgress(value=0.0, description='%')

interactive(children=(Dropdown(description='Shima_T_fz', options=('Niemand_et_al_2012', 'Bigg_1953'), value='N…